In [ ]:
#pyqt로 작성
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5 import QtCore, QtGui, QtWidgets 
import cv2
import time
import numpy as np
import datetime
from tkinter import *
import sys
from os import path

#UI파일 연결
#단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
form_class = uic.loadUiType("controller.ui")[0]
c = 0
s = 0
q = 0

record = False
rs = False
r = False

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :

    def __init__(self) :

        super().__init__()
        self.setupUi(self)

        #버튼에 기능을 연결하는 코드
        self.pushButton.clicked.connect(self.Webcams)
        self.pushButton_2.clicked.connect(self.Backs)
        self.pushButton_3.clicked.connect(self.Nexts)
        self.pushButton_4.clicked.connect(self.Resets)
        self.pushButton_5.clicked.connect(self.Captures)
        self.pushButton_6.clicked.connect(self.Records)
        self.pushButton_7.clicked.connect(self.Stop_Records)
        self.pushButton_8.clicked.connect(self.Exit)

    def Webcams(self):
        global s
        global q
        global record
        global rs
        global r

        q = 0

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        record = False

        logo = cv2.imread("./zero.png")

        #logo = cv2.resize(logo, (275,183))

        #logo_height, logo_width, _ = logo.shape

        cap = cv2.VideoCapture(0)

        cap.set(10000, 10000)

        #cap_height = cv2.CAP_PROP_FRAME_HEIGHT
        #cap_width =  cv2.CAP_PROP_FRAME_WIDTH

        time.sleep(3) #warming up
        if not cap.isOpened():
            exit()

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        while True:    
            ret, image = cap.read()

            #현재시간
            now = datetime.datetime.now().strftime("%d_%H-%M-%S")
            key = cv2.waitKey(3)

            #대체 이미지
            if c == 0: 
                logo = cv2.imread("./zero.png")
            elif c == 2: 
                logo = cv2.imread("./pick.png")
            elif c == 3: 
                logo = cv2.imread("./smile.png")
            elif c == 4:
                logo = cv2.imread("./keobi.png")
            elif c == 5:
                logo = cv2.imread("./hahoi.png")
            elif c == 6:
                logo = cv2.imread("./trumph.png")

            faces = face_cascade.detectMultiScale(image)

            for face in faces:
                (x, y, w, h) = face
                x1, y1, x2, y2 = x, y, x + w, y + h
                #cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)

                if(c == 1):
                    image_face = image[y1:y2, x1:x2]
                    image_face = cv2.resize(image_face, (w//30, h//30))
                    image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                    image[y1:y2, x1:x2] = image_face

                else:
                    image_face = image[y1:y2, x1:x2]
                    #logo = cv2.resize(logo, (y1-y2, x1-x2))
                    #image_face = logo[y1:y2, x1:x2]
                    logo = cv2.resize(logo, (w, h))
                    image_face =  cv2.add(image_face, logo)
                    image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                    image[y1:y2, x1:x2] = image_face


                if(s == 1): #z
                    #print("캡쳐")
                    cv2.imwrite("./" + str(now) + ".png", image)
                    s = 0

                if(record == True):
                    record = False
                    r = True
                    #print("녹화 시작")
                    video = cv2.VideoWriter("./" + str(now) + ".avi", fourcc, 30.0, (image.shape[1], image.shape[0]))

                elif((r == True) and (rs == True)):
                    #print("녹화 중지")
                    video.release()
                    record = False
                    rs = False
                    r = False

                if(r == True):    
                    color = (0, 0, 255)
                    thickness = 2 
                    location = (550, 50)
                    font = cv2.FONT_HERSHEY_COMPLEX  # hand-writing style font
                    fontScale = 1.0
                    cv2.putText(image, 'REC', location, font, fontScale, color, thickness)
                    #print("녹화 중..")
                    video.write(image)


                cv2.imshow("result", image)


            if (q == 1): #q
                break

            if not ret:
                break

            #image = cv2.add(image, logo)

        cap.release()
        cv2.destroyAllWindows()

    def Nexts(self):
        global c
        if(c == 6):
            c = c
        else:
            c = c + 1

    def Backs(self):
        global c
        if(c == 0):
            c = 0
        else:
            c = c - 1  

    def Resets(self):
        global c
        c = 0

    def Captures(self):
        global s
        s = 1

    def Records(self):
        global record
        global r
        record = True
        r = True

    def Stop_Records(self):
        global rs
        rs = True

    def Exit(self):
        global q
        q = 1


if __name__ == "__main__" :
    app = QApplication(sys.argv)
    myWindow = WindowClass() 
    myWindow.show()
    app.exec_()

In [1]:
#pyqt로 작성
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5 import QtCore, QtGui, QtWidgets 
import cv2
import time
import numpy as np
import datetime
from tkinter import *
from PyQt5 import QtCore
from PyQt5 import QtWidgets
from PyQt5 import QtGui
import sys
from os import path
from flask import Flask

app = Flask(__name__)

@app.route('/') 
def home(): 
    return '''
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <title>Web Cam Video</title>
                <link rel="stylesheet" href="./stylesheets/main.css">
            <style>
            H1 {
              position: absolute;
              left: 100px;
              top: 15px;
              color:black;
            }
            </style>


            <H1>Webcam</H1>
            </head>
            <body>
                  <div class="booth">
                      <video id="video" width="400" height="300" autoplay=""></video>
                      <canvas id="canvas" width="400" height="300"></canvas>
            </div>
                  <script src="./javascripts/video.js"></script>
            </body>
            </html>

            <style>
            H2 {
              position: absolute;
              left: 130px;
              top: 100px;
              color:red;
            }
            </style>


            <style>
            H3 {
              position: absolute;
              left: 300px;
              top: 200px;
              color:red;
            }
            </style>

            <style>
            H4 {
              position: absolute;
              left: 300px;
              top: 250px;
              color:red;
            }
            </style>


            <style>
            H5 {
              position: absolute;
              left: 100px;
              top: 300px;
              color:red;
            }
            </style>


            <style>
            H6 {
              position: absolute;
              left: 200px;
              top: 297px;
              color:red;
            }
            </style>

            <H2><form action="/webcam/" method="post"><button name="forwardBtn" type="submit">webcam</button></H2></form>'''

@app.route("/webcam/", methods=['GET', 'POST'])
def Web_cam():
    #UI파일 연결
    #단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
    form_class = uic.loadUiType("controller.ui")[0]
    c = 0
    s = 0
    q = 0

    record = False
    rs = False
    r = False

    #화면을 띄우는데 사용되는 Class 선언
    class WindowClass(QMainWindow, form_class) :

        def __init__(self) :

            super().__init__()
            self.setupUi(self)

            #버튼에 기능을 연결하는 코드
            self.pushButton.clicked.connect(self.Webcams)
            self.pushButton_2.clicked.connect(self.Backs)
            self.pushButton_3.clicked.connect(self.Nexts)
            self.pushButton_4.clicked.connect(self.Resets)
            self.pushButton_5.clicked.connect(self.Captures)
            self.pushButton_6.clicked.connect(self.Records)
            self.pushButton_7.clicked.connect(self.Stop_Records)
            self.pushButton_8.clicked.connect(self.Exit)

        def Webcams(self):
            global c
            global s
            global q
            global record
            global rs
            global r

            q = 0
            c = 0
            s = 0
            
            record = False
            rs = False
            r = False

            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            record = False

            logo = cv2.imread("./zero.png")

            #logo = cv2.resize(logo, (275,183))

            #logo_height, logo_width, _ = logo.shape

            cap = cv2.VideoCapture(0)

            cap.set(10000, 10000)

            #cap_height = cv2.CAP_PROP_FRAME_HEIGHT
            #cap_width =  cv2.CAP_PROP_FRAME_WIDTH

            time.sleep(3) #warming up
            if not cap.isOpened():
                exit()

            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

            while True:    
                ret, image = cap.read()

                #현재시간
                now = datetime.datetime.now().strftime("%d_%H-%M-%S")
                key = cv2.waitKey(3)

                #대체 이미지
                if c == 0: 
                    logo = cv2.imread("./zero.png")
                elif c == 2: 
                    logo = cv2.imread("./pick.png")
                elif c == 3: 
                    logo = cv2.imread("./smile.png")
                elif c == 4:
                    logo = cv2.imread("./keobi.png")
                elif c == 5:
                    logo = cv2.imread("./hahoi.png")
                elif c == 6:
                    logo = cv2.imread("./trumph.png")

                faces = face_cascade.detectMultiScale(image)

                for face in faces:
                    (x, y, w, h) = face
                    x1, y1, x2, y2 = x, y, x + w, y + h
                    #cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
                    
                    if(c == 1):
                        image_face = image[y1:y2, x1:x2]
                        image_face = cv2.resize(image_face, (w//30, h//30))
                        image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                        image[y1:y2, x1:x2] = image_face
                        
                    else:
                        image_face = image[y1:y2, x1:x2]
                        #logo = cv2.resize(logo, (y1-y2, x1-x2))
                        #image_face = logo[y1:y2, x1:x2]
                        logo = cv2.resize(logo, (w, h))
                        image_face =  cv2.add(image_face, logo)
                        image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                        image[y1:y2, x1:x2] = image_face

                    if(s == 1): #z
                        #print("캡쳐")
                        cv2.imwrite("./" + str(now) + ".png", image)
                        s = 0

                    if(record == True):
                        record = False
                        r = True
                        #print("녹화 시작")
                        video = cv2.VideoWriter("./" + str(now) + ".avi", fourcc, 30.0, (image.shape[1], image.shape[0]))

                    elif((r == True) and (rs == True)):
                        #print("녹화 중지")
                        video.release()
                        record = False
                        rs = False
                        r = False

                    if(r == True):    
                        color = (0, 0, 255)
                        thickness = 2 
                        location = (550, 50)
                        font = cv2.FONT_HERSHEY_COMPLEX  # hand-writing style font
                        fontScale = 1.0
                        cv2.putText(image, 'REC', location, font, fontScale, color, thickness)
                        #print("녹화 중..")
                        video.write(image)


                    cv2.imshow("result", image)


                if (q == 1): #q
                    break

                if not ret:
                    break

                #image = cv2.add(image, logo)

            cap.release()
            cv2.destroyAllWindows()

        def Nexts(self):
            global c
            if(c == 6):
                c = c
            else:
                c = c + 1

        def Backs(self):
            global c
            if(c == 0):
                c = 0
            else:
                c = c - 1  

        def Resets(self):
            global c
            c = 0

        def Captures(self):
            global s
            s = 1

        def Records(self):
            global record
            global r
            record = True
            r = True

        def Stop_Records(self):
            global rs
            rs = True

        def Exit(self):
            global q
            q = 1


    if __name__ == "__main__" :
        app = QApplication(sys.argv)
        myWindow = WindowClass() 
        myWindow.show()
        app.exec_()
        
    return home()
        
if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Apr/2020 12:33:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:33:43] "GET /stylesheets/main.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2020 12:33:43] "GET /javascripts/video.js HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2020 12:34:08] "POST /webcam/ HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:34:08] "GET /webcam/stylesheets/main.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2020 12:34:08] "GET /webcam/javascripts/video.js HTTP/1.1" 404 -
